# PHT cohort counter

In [52]:
import requests
import json
import uuid
signalStation = "http://vwdata.personalhealthtrain.net"

def getStations(signalStation):
    response = requests.get(signalStation + "/client")
    return json.loads(response.text)

def defineTrain(image, inputVars, runId):
    return {
        "image": image,
        "inputString": ("%s" % json.dumps(inputVars)),
        "runId": runId
    }

def requestExecution(signalStation, clientId, taskDescription):
    response = requests.post(signalStation + "/client/" + str(clientId) + "/task/add", json=taskDescription)
    taskId = json.loads(response.text)["taskId"]
    return taskId

## List Stations

In [49]:
getStations(signalStation)

[{'id': 1,
  'name': 'The Maastricht Study',
  'email': 'johan.vansoest@maastro.nl',
  'institute': 'Maastricht University',
  'country': 'The Netherlands',
  'ip': '192.168.96.1',
  'last_seen': '2019-02-21 11:49:23.147569'},
 {'id': 2,
  'name': 'VWData Station Maastro',
  'email': 'johan.vansoest@maastro.nl',
  'institute': 'MAASTRO Clinic',
  'country': 'Netherlands',
  'ip': '192.168.96.1',
  'last_seen': '2019-02-21 08:47:10.060424'}]

## Define train

In [50]:
inputData = {
    "stage": 1
}

In [53]:
taskMetaData = defineTrain("fairhealth/ppdml", inputData, 1)
print(taskMetaData)

{'image': 'fairhealth/ppdml', 'inputString': '{"stage": 1}', 'runId': 1}


## Run cohort counter on RDF station

In [60]:
clientId = 1
taskId = requestExecution(signalStation, clientId, taskMetaData)
print("Task submitted with ID: %s" % taskId)

Task submitted with ID: 14


### Receive cohort counter result from station

In [37]:
response = requests.get(signalStation + "/client/" + str(clientId) + "/task/" + str(taskId) + "/result")
print(json.loads(response.text)[0]["log"])
#json.loads(json.loads(response.text)[0]["response"])


Traceback (most recent call last):
  File "run.py", line 19, in <module>
    currentStage = inputArgs["stage"]
TypeError: string indices must be integers

=======================INTERNAL FILE LOG===================== 



## Run cohort counter on FHIR station

In [ ]:
clientId = 1

# Post train on signal station
response = requests.post(signalStation + "/client/" + str(clientId) + "/task/add", json=taskMetaData)
taskId = json.loads(response.text)["taskId"]

print("Task submitted with ID: %s" % taskId)

### Receive cohort counter result from station

In [ ]:
response = requests.get(signalStation + "/client/" + str(clientId) + "/task/" + str(taskId) + "/result")
json.loads(json.loads(response.text)[0]["response"])